<a href="https://colab.research.google.com/github/Jerryson520/NLP-Projects/blob/main/Spam%20Binary%20classification%20with%20GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install torchmetrics

In [25]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random
from torchmetrics.classification import Recall, Accuracy, AUROC, Precision

In [26]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2024-05-24 18:43:10--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.2’

smsspamcollection.z     [   <=>              ] 198.65K   309KB/s    in 0.6s    

2024-05-24 18:43:11 (309 KB/s) - ‘smsspamcollection.zip.2’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [27]:
!unzip -o smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [28]:
!head -10 SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [29]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2024-05-24 18:43:12--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.3’

smsspamcollection.z     [   <=>              ] 198.65K   309KB/s    in 0.6s    

2024-05-24 18:43:13 (309 KB/s) - ‘smsspamcollection.zip.3’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [30]:
file_path = 'SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index = [])
with open(file_path) as f:
    for line in f.readlines():
        split = line.split('\t')
        df = pd.concat([
                df,
                pd.DataFrame.from_dict({
                    'label': [1 if split[0] == 'spam' else 0],
                    'text': [split[1]]
                })
            ],
            ignore_index=True
        )
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [31]:
text = df.text.values
labels = df.label.values

In [32]:
# Set to the GPT2Tokenizer and set lower case to True
tokenizer = GPT2Tokenizer.from_pretrained(
    'gpt2',
    do_lower_case=True)
tokenizer.pad_token = tokenizer.eos_token
# Set the padding to 'left' or 'right'?
# Remember we want to use the last token's embedding to represent the entire sentence
tokenizer.padding_side = 'left'

In [33]:
def print_rand_sentence():
    '''Displays the tokens and respective IDs of a random text sample'''
    index = random.randint(0, len(text)-1)
    print(text[index])
    table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

Hi Chachi tried calling u now unable to reach u .. Pl give me a missed cal once u c tiz msg  Kanagu

╒══════════╤═════════════╕
│ Tokens   │   Token IDs │
╞══════════╪═════════════╡
│ Hi       │       17250 │
├──────────┼─────────────┤
│ ĠCh      │         609 │
├──────────┼─────────────┤
│ achi     │       14299 │
├──────────┼─────────────┤
│ Ġtried   │        3088 │
├──────────┼─────────────┤
│ Ġcalling │        4585 │
├──────────┼─────────────┤
│ Ġu       │         334 │
├──────────┼─────────────┤
│ Ġnow     │         783 │
├──────────┼─────────────┤
│ Ġunable  │        5906 │
├──────────┼─────────────┤
│ Ġto      │         284 │
├──────────┼─────────────┤
│ Ġreach   │        3151 │
├──────────┼─────────────┤
│ Ġu       │         334 │
├──────────┼─────────────┤
│ Ġ..      │       11485 │
├──────────┼─────────────┤
│ ĠPl      │        1345 │
├──────────┼─────────────┤
│ Ġgive    │        1577 │
├──────────┼─────────────┤
│ Ġme      │         502 │
├──────────┼─────────────┤
│ Ġa    

In [34]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  # Use the tokenizer and the encode_plus methods to return the right data we'll need
  # Set max_length = 32 and return_tokens = 'pt'
  # Set other fields to the appropriate booleans needed
  return tokenizer.encode_plus(
      input_text,
      max_length=32,
      pad_to_max_length=True,
      return_tensors='pt',
      return_token_type_ids=True,
      return_attention_mask=True)


for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])


# Gather all the token_id, attention masks, and labels
token_id = torch.cat(token_id, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [35]:
token_id

tensor([[50256, 50256, 50256,  ...,  4383,   986,   198],
        [50256, 50256, 50256,  ...,    72,   986,   198],
        [11146,  5726,   287,  ...,  5726,  1808,     7],
        ...,
        [50256, 50256, 50256,  ..., 11776,    30,   198],
        [50256, 50256, 50256,  ...,   329,  1479,   198],
        [50256, 50256, 50256,  ...,   663,  1438,   198]])

In [36]:
def print_rand_sentence_encoding():
    '''Displays tokens, token IDs and attention mask of a random text sample'''
    index = random.randint(0, len(text) - 1)
    tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
    print(tokens)
    token_ids = [i.numpy() for i in token_id[index]]
    attention = [i.numpy() for i in attention_masks[index]]
    table = np.array([tokens, token_ids, attention]).T
    print(
        tabulate(
            table,
            headers = ['Tokens', 'Token IDs', 'Attention Mask'],
            tablefmt = 'fancy_grid')
    )

print_rand_sentence_encoding()

['<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', 'I', "'ll", 'Ġtalk', 'Ġto', 'Ġthe', 'Ġothers', 'Ġand', 'Ġprobably', 'Ġjust', 'Ġcome', 'Ġearly', 'Ġtomorrow', 'Ġthen', 'Ċ']
╒═══════════════╤═════════════╤══════════════════╕
│ Tokens        │   Token IDs │   Attention Mask │
╞═══════════════╪═════════════╪══════════════════╡
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endof

In [37]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
# Use train_test_split
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size=val_ratio,
    shuffle=True,
    stratify=labels)

# Train and validation sets
# Set to TensorDataset
train_set = TensorDataset(
    token_id[train_idx],
    attention_masks[train_idx],
    labels[train_idx])

val_set = TensorDataset(
    token_id[val_idx],
    attention_masks[val_idx],
    labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
    train_set,
    batch_size=batch_size,
    sampler=RandomSampler(train_set))

validation_dataloader = DataLoader(
    val_set,
    batch_size=batch_size,
    sampler=RandomSampler(val_set))

### Load specific versions of the model

In [38]:
# Load the GPT2ForSequenceClassification model
# Do not ouput the attentions and all hidden states

config = GPT2Config()

# Set to 'gpt2' (the smallest GPT2 which is 120 M parameters)
# Use the config above and set other labels as needed
model = GPT2ForSequenceClassification(config)

# Set the pad token id to the eos token id
model.config.pad_token_id = model.config.eos_token_id

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5
# See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 5e-5,
    eps = 1e-08
)

### Set the model to the right device

In [39]:
# If on GPU, do as below
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [40]:
model = model.to(device)

# Recommended number of epochs: See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

In [41]:
# Print all the layers of this GPT2 model and the number of parameters per layer
# If this is False, fine tune just the classifier layer and leave all other GPT2 parameters alone
# If this is True, fine tune everything
fine_tune = True

# Turn off gradients using the above
if fine_tune:
  for param in model.parameters():
    param.requires_grad=True
else:
  for param in model.base_model.parameters():
    param.requires_grad=False

total_parameters=0
for name, param in model.named_parameters():
  print(f"Name: {name}, Parameters: {param}")
  total_parameters += param.numel()

assert(total_parameters == 124441344)

Name: transformer.wte.weight, Parameters: Parameter containing:
tensor([[-0.0282,  0.0175,  0.0165,  ..., -0.0183, -0.0287,  0.0289],
        [-0.0011, -0.0220,  0.0201,  ..., -0.0269,  0.0359, -0.0213],
        [-0.0122,  0.0110,  0.0208,  ..., -0.0440, -0.0043, -0.0172],
        ...,
        [ 0.0126,  0.0096, -0.0155,  ..., -0.0305,  0.0147,  0.0022],
        [ 0.0139, -0.0118,  0.0112,  ..., -0.0246, -0.0536, -0.0164],
        [ 0.0078,  0.0012, -0.0003,  ...,  0.0381,  0.0350,  0.0133]],
       device='cuda:0', requires_grad=True)
Name: transformer.wpe.weight, Parameters: Parameter containing:
tensor([[ 0.0411,  0.0124,  0.0005,  ..., -0.0004,  0.0001, -0.0007],
        [-0.0258, -0.0344, -0.0045,  ...,  0.0283,  0.0060,  0.0498],
        [ 0.0092, -0.0169, -0.0079,  ...,  0.0280,  0.0155, -0.0112],
        ...,
        [ 0.0523,  0.0253,  0.0026,  ..., -0.0071,  0.0131, -0.0180],
        [-0.0155, -0.0116,  0.0193,  ..., -0.0053, -0.0226, -0.0410],
        [ 0.0124, -0.0314, -0.0

### Train the model

In [42]:
num_classes = 2

# Use torchmetrics to set up accuracy, recall, precision, and auroc
accuracy = Accuracy(task='binary', num_classes=num_classes)
recall = Recall(task='binary', num_classes=num_classes)
precision = Precision(task='binary', num_classes=num_classes)
auroc = AUROC(task='binary', num_classes=num_classes)

In [46]:
# Main training / validation loop
for _ in trange(epochs, desc = 'Epoch'):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        # Put each element of batch onto the device
        batch = tuple(t.to(device) for t in batch)

        # Unpack the batch
        input_ids, attention_mask, labels = batch

        # Set gradients to zero
        optimizer.zero_grad()

        # Forward pass
        train_output = model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels)

        # Backward pass
        train_output.loss.backward()
        optimizer.step()

        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += input_ids.size()[0]
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)

        # Unpack the batch
        input_ids, attention_mask, labels = batch

        with torch.no_grad():
          # Forward pass
            eval_output = model(
                input_ids,
                attention_mask=attention_mask)

        # Calculate validation metrics
        labels = labels.detach().cpu()
        predicted_labels = eval_output.logits.argmax(dim=-1).detach().cpu()

        val_accuracy.append(accuracy(predicted_labels,labels))
        val_recall.append(recall(predicted_labels,labels))
        val_precision.append(precision(predicted_labels,labels))
        val_auroc.append(auroc(predicted_labels,labels))

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
Epoch:  50%|█████     | 1/2 [00:24<00:24, 24.41s/it]


	 - Train loss: 0.1678
	 - Validation Accuracy: 0.9839
	 - Validation Precision: 0.8952
	 - Validation Recall: 0.8126
	 - Validation AUROC: 0.8629



Epoch: 100%|██████████| 2/2 [00:48<00:00, 24.46s/it]


	 - Train loss: 0.0286
	 - Validation Accuracy: 0.9607
	 - Validation Precision: 0.7643
	 - Validation Recall: 0.8995
	 - Validation AUROC: 0.8957



### Test on a specific sentence, see the outcome

In [47]:
new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Spam' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Ham'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Predicted Class:  Spam


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


### Questions

Question 1: Run the above by fine tuning GPT2 and the classfier head and by not doing this (using GPT2 as a feature encoder). What is the gap between this? What are the metrics we get in each case?

Solution: FILL Paste your output here